# Library Download and Load

In [ ]:

!pip install transformers
!pip install pytorch-lightning
!pip install sentencepiece
!pip install bertviz
!pip install plotly
!pip install pyyaml==5.4.1

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 69.8 MB/s 
     |████████████████████████████████| 3.3 MB 47.5 MB/s 
     |████████████████████████████████| 59 kB 8.3 MB/s 
     |████████████████████████████████| 895 kB 58.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 329 kB 65.1 MB/s 
     |████████████████████████████████| 829 kB 66.1 MB/s 
     |████████████████████████████████| 132 kB 84.3 MB/s 
     |████████████████████████████████| 1.1 MB 74.4 MB/s 
     |████████████████████████████████| 192 kB 93.5 MB/s 
     |████████████████████████████████| 271 kB 93.5 MB/s 
     |████████████████████████████████| 160 kB 88.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=7dadf930308459

In [ ]:

import json
import pandas as pd
import numpy as np
import torch
torch.cuda.empty_cache()
from pathlib import Path # 파일시스템의 경로를 단순한 문자열이 아닌 객체로 다루게 하는 것
from torch.utils.data import Dataset, DataLoader, SequentialSampler
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from termcolor import colored
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim



from transformers import (
    AdamW,
    RobertaForSequenceClassification,
    RobertaModel,
    RobertaTokenizer,
    RobertaForQuestionAnswering,
    AutoModel,
    AutoTokenizer,
    BertTokenizerFast, AlbertModel,
    AlbertTokenizer,
    AutoModelForQuestionAnswering
)
from tqdm.auto import tqdm
from bertviz import head_view

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import gc


import warnings

warnings.filterwarnings(action='ignore') 
from plotly import graph_objs as go

from collections import Counter
import plotly.express as px

In [ ]:
pl.seed_everything(42)
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()

Global seed set to 42


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

DIR = "/content/drive/MyDrive/Colab Notebooks/goormtextclassificationproject/goormkoreanmrcproject/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())



with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [ ]:
train = pd.DataFrame(columns=['guid' ,'title' ,'news_category', 'context', 'question' ,'answer_start', 'text'])
guid = 0
for data in TRAIN_DATA['data']:
    for paragraphs in data['paragraphs']:
        for line in paragraphs['qas']:
            train.loc[guid, 'context'] = paragraphs['context']
            train.loc[guid, 'question'] = line['question']
            train.loc[guid, 'answer_start'] = line['answers'][0]['answer_start']
            train.loc[guid, 'text'] = line['answers'][0]['text']            
            train.loc[guid, 'title'] = data['title']
            train.loc[guid, 'guid'] = line['guid']
            train.loc[guid, 'news_category'] = data['news_category']
            guid += 1

In [ ]:
test = pd.DataFrame(columns=['guid','title' ,'news_category', 'context', 'question' ])
guid = 0
for data in TEST_DATA['data']:
    for paragraphs in data['paragraphs']:
        for line in paragraphs['qas']:
            test.loc[guid, 'context'] = paragraphs['context']
            test.loc[guid, 'question'] = line['question']
            test.loc[guid, 'title'] = data['title']
            test.loc[guid, 'guid'] = line['guid']
            test.loc[guid, 'news_category'] = data['news_category']
            guid += 1

In [ ]:
# train = train[:100] # test

In [ ]:
train_ai_hub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/goormtextclassificationproject/goormkoreanmrcproject/Korean MRC/data/ai_hub_data.csv')

In [ ]:
train = train.drop('news_category', axis=1)
train = pd.concat([train, train_ai_hub]).reset_index().drop('index', axis=1)
train = train.reset_index().drop('index', axis=1)

In [ ]:

train = train[:30000] # version_1은 사용하지 않고, version_2 : 30000

In [ ]:

def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }


train['answers'] = train[['answer_start', 'text']].apply(get_answers, axis=1)

In [ ]:

train_df, val_df = train_test_split(train, test_size=0.1, random_state=42)

train_df = train_df.reset_index().drop('index', axis=1)
val_df = val_df.reset_index().drop('index', axis=1)

train_df.shape, val_df.shape

((45000, 7), (5000, 7))

In [ ]:
max_length = 1024

In [ ]:

MODEL_NAME = 'monologg/kobigbird-bert-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/373 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/169 [00:00<?, ?B/s]

In [ ]:
def prepare_train_features(example, tokenizer):
    example["question"] = example["question"].lstrip()
    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=max_length,
        #stride=doc_stride, 
        return_overflowing_tokens=True, # 길이를 넘어가는 토큰들을 반환할 것인지
        return_offsets_mapping=True,  # 각 토큰에 대해 (char_start, char_end) 정보를 반환한 것인지
        padding="max_length",
    )

    sample_mapping = tokenized_example.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_example.pop("offset_mapping")

    features = []
    for i, offsets in enumerate(offset_mapping):
        feature = {}

        input_ids = tokenized_example["input_ids"][i]
        attention_mask = tokenized_example["attention_mask"][i]

        feature['input_ids'] = input_ids
        feature['attention_mask'] = attention_mask
        feature['offset_mapping'] = offsets

        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_example.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = example["answers"]

        if len(answers["answer_start"]) == 0:
            feature["start_position"] = cls_index
            feature["end_position"] = cls_index
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                feature["start_position"] = cls_index
                feature["end_position"] = cls_index
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                feature["start_position"] = token_start_index - 1
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                feature["end_position"] = token_end_index + 1

        features.append(feature)
    return features

In [ ]:
train_features, valid_features = [[] for _ in range(2)]
for i, row in tqdm(train_df.iterrows()):
    train_features += prepare_train_features(row, tokenizer)
for i, row in val_df.iterrows():
    valid_features += prepare_train_features(row, tokenizer)

0it [00:00, ?it/s]

10989

In [ ]:
class DatasetRetriever(Dataset):
    def __init__(self, features, mode='train'):
        super(DatasetRetriever, self).__init__()
        self.features = features
        self.mode = mode
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, item):   
        feature = self.features[item]
        if self.mode == 'train':
            return {
                'input_ids':torch.tensor(feature['input_ids'] ,dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':torch.tensor(feature['offset_mapping'], dtype=torch.long),
                'start_position':torch.tensor(feature['start_position'], dtype=torch.long),
                'end_position':torch.tensor(feature['end_position'], dtype=torch.long)
            }
        else:
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':feature['offset_mapping'],
                'sequence_ids':feature['sequence_ids'],
                'id':feature['example_id'],
                'context': feature['context'],
                'question': feature['question']
            }

In [ ]:
train_dataset = DatasetRetriever(train_features)

In [ ]:
for data in train_dataset:
  print("Input ids: ", data['input_ids'][:10])
  print("start_positions: ", data['start_position'])
  print("end_positions: ", data['end_position'])
  break

Input ids:  tensor([    2,  9751,  7953, 31318,  6852, 10512,  4568,  5189,  4741, 12213])
start_positions:  tensor(486)
end_positions:  tensor(502)


In [ ]:
class KorQuadModule(pl.LightningDataModule):
  def __init__(
      self, 
      train_features,
      valid_features,
      tokenizer: tokenizer,
      batch_size: int = 4,
      source_max_token_len: int = 1024
  ):
    super().__init__()
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.source_max_token_len = source_max_token_len
  def setup(self):

    self.train_dataset = DatasetRetriever(train_features)
    self.valid_dataset = DatasetRetriever(valid_features)
    

  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=4
      
    )
  def val_dataloader(self):
    return DataLoader(
        self.valid_dataset,
        batch_size=self.batch_size,
        num_workers=4
    )
  def test_dataloader(self):
    return DataLoader(
        self.valid_dataset,
        batch_size=1,
        num_workers=4 

    )

In [ ]:

BATCH_SIZE = 4
N_EPOCHS = 10
data_module = KorQuadModule(train_features, valid_features, tokenizer, batch_size=BATCH_SIZE)
data_module.setup() 


In [ ]:
class KorQuadModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
  
    self.xlm_roberta  = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME, output_hidden_states=True)
    #self.qa_outputs = nn.Linear(self.xlm_roberta.config.hidden_size, 2)
    #self.dropout = nn.Dropout(0.1)
    #self._init_weights(self.qa_outputs)
    #self.cnn =  nn.Conv1d(self.xlm_roberta.config.hidden_size*3, self.xlm_roberta.config.hidden_size, 3, padding=1)
    #self.gelu = nn.GELU()

    #self.se_head = nn.Linear(self.xlm_roberta.config.hidden_size, 2)
    #self.inst_head = nn.Linear(self.xlm_roberta.config.hidden_size, 2)
      
  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      module.weight.data.normal_(mean=0.0, std=0.02)
      if module.bias is not None:
        module.bias.data.zero_()

  def forward(self, input_ids, attention_mask):
    output = self.xlm_roberta(input_ids, attention_mask)
    #hs = output['hidden_states']
    #pooled_output = output['pooler_output'] 
    #sequence_output = outputs[0]
    #pooled_output = outputs[1]    
    # sequence_output = self.dropout(sequence_output)
    # qa_logits = self.qa_outputs(sequence_output)

    #sequence_output = self.gelu(self.cnn(sequence_output.permute(0,2,1)).permute(0,2,1))
    #se_out = self.se_head(self.dropout(sequence_output))  #()    
    
    #start_logits, end_logits = se_out.split(1, dim=-1)
    #start_logits = start_logits.squeeze(-1)
    #end_logits = end_logits.squeeze(-1)
    

    #seq_output = torch.cat([hs[-1],hs[-2],hs[-3]], dim=-1)

    # seq_output = hs[-1]

    # avg_output = torch.sum(seq_output*masks.unsqueeze(-1), dim=1, keepdim=False)
    # avg_output = avg_output/torch.sum(masks, dim=-1, keepdim=True)
    # +max_output
    #whole_out = self.whole_head(avg_output)

    #seq_output = self.gelu(self.cnn(seq_output.permute(0,2,1)).permute(0,2,1))
        
    #se_out = self.se_head(self.dropout(seq_output))  #()
    #start_logits, end_logits = se_out.split(1, dim=-1)
    #return se_out[:, :, 0], se_out[:, :, 1]
    start_logits = output.start_logits
    end_logits = output.end_logits

    return start_logits, end_logits

  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_label = batch['start_position'].to(device)
    end_label = batch['end_position'].to(device)
    
    start_pred, end_pred = self(input_ids, attention_mask)

    start_loss = nn.CrossEntropyLoss(ignore_index=-1)(start_pred, start_label)
    end_loss = nn.CrossEntropyLoss(ignore_index=-1)(end_pred, end_label)
    loss = (start_loss + end_loss) / 2
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_label = batch['start_position'].to(device)
    end_label = batch['end_position'].to(device)
    
    start_pred, end_pred = self(input_ids, attention_mask)

    start_loss = nn.CrossEntropyLoss(ignore_index=-1)(start_pred, start_label)
    end_loss = nn.CrossEntropyLoss(ignore_index=-1)(end_pred, end_label)
    loss = (start_loss + end_loss) / 2
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss


  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_label = batch['start_position'].to(device)
    end_label = batch['end_position'].to(device)
    
    start_pred, end_pred = self(input_ids, attention_mask)

    start_loss = nn.CrossEntropyLoss(ignore_index=-1)(start_pred, start_label)
    end_loss = nn.CrossEntropyLoss(ignore_index=-1)(end_pred, end_label)
    loss = (start_loss + end_loss) / 2
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def predict_step(self, batch, batch_idx, dataloader_idx=0):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)

    #start_pred, end_pred = self(input_ids, attention_mask)

    return self(input_ids, attention_mask)


  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=0.00005) # 0.0001 -> 0.00005
    return optimizer

In [ ]:
model = KorQuadModel()

Downloading:   0%|          | 0.00/870 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'bert.pooler.weight', 'bert.pooler.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BigBirdForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of 

In [ ]:
logger = TensorBoardLogger("lightning_logs", name='Korquad_log')
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

checkpoint_callback = ModelCheckpoint(
    filename="/content/drive/MyDrive/Colab Notebooks/goormtextclassificationproject/goormkoreanmrcproject/model",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min")

In [ ]:
trainer = pl.Trainer(
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping_callback],
    max_epochs=10,
    gpus=1,
    #tpu_cores=8,
    progress_bar_refresh_rate=30)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type                        | Params
------------------------------------------------------------
0 | xlm_roberta | BigBirdForQuestionAnswering | 117 M 
------------------------------------------------------------
117 M     Trainable params
0         Non-trainable params
117 M     Total params
471.555   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 11517: val_loss reached 6.06009 (best 6.06009), saving model to "/content/drive/MyDrive/Colab Notebooks/goormtextclassificationproject/goormkoreanmrcproject/model-v3.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Epoch 1, global step 23035: val_loss was not in top 1


In [ ]:

%load_ext tensorboard
%tensorboard --logdir ./lightning_logs
#!rm --rf lightning_logs
#/content/lightning_logs

In [ ]:

trained_model = KorQuadModel().load_from_checkpoint("/content/drive/MyDrive/Colab Notebooks/goormtextclassificationproject/goormkoreanmrcproject/model-v3.ckpt")
trained_model.freeze()

Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'bert.pooler.weight', 'bert.pooler.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BigBirdForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of 

In [ ]:
gc.collect()

3641

## Inference

In [ ]:

def prepare_test_features(example, tokenizer):
    example["question"] = example["question"].lstrip()
    
    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=1024,
        #stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    features = []
    for i in range(len(tokenized_example["input_ids"])):
        feature = {}
        feature["example_id"] = example['guid']
        feature['context'] = example['context']
        feature['question'] = example['question']
        feature['input_ids'] = tokenized_example['input_ids'][i]
        feature['attention_mask'] = tokenized_example['attention_mask'][i]
        feature['offset_mapping'] = tokenized_example['offset_mapping'][i]
        feature['sequence_ids'] = [0 if i is None else i for i in tokenized_example.sequence_ids(i)]
        features.append(feature)
    return features

In [ ]:
import collections

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 20):
    all_start_logits, all_end_logits = raw_predictions
    
    example_id_to_index = {k: i for i, k in enumerate(examples["guid"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in examples.iterrows():
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            sequence_ids = features[feature_index]["sequence_ids"]
            context_index = 1

            features[feature_index]["offset_mapping"] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(features[feature_index]["offset_mapping"])
            ]
            offset_mapping = features[feature_index]["offset_mapping"]
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["guid"]] = best_answer["text"]
        
        
    return predictions

In [ ]:

test_features = []
for i, row in test.iterrows():
    test_features += prepare_test_features(row, tokenizer)

In [ ]:

test_dataset = DatasetRetriever(test_features, mode='test')
test_dataloader = DataLoader(
    test_dataset,
    batch_size=4, 
    sampler=SequentialSampler(test_dataset),
    num_workers=4,
    pin_memory=True, 
    drop_last=False
)

In [ ]:
def get_predictions(trained_model):
    model = trained_model.to(device)
    
    start_logits = []
    end_logits = []
    for batch in tqdm(test_dataloader):
        with torch.no_grad():
            outputs_start, outputs_end = model(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
            start_logits.append(outputs_start.cpu().numpy().tolist())
            end_logits.append(outputs_end.cpu().numpy().tolist())
            del outputs_start, outputs_end
    gc.collect()
    return np.vstack(start_logits), np.vstack(end_logits)

In [ ]:
start_logits, end_logits = get_predictions(trained_model=trained_model)
# Now Calling Function 

fin_preds = postprocess_qa_predictions(test, test_features, (start_logits, end_logits))

In [ ]:
from string import punctuation
submission = []
for p1, p2 in fin_preds.items():
    p2 = " ".join(p2.split())
    p2 = p2.strip(punctuation)
    submission.append((p1, p2))
    
sample = pd.DataFrame(submission, columns=["id", "Predicted"])

#test_data =pd.merge(left=test_df,right=sample,on='id')

In [ ]:
sample.to_csv('/content/drive/MyDrive/Colab Notebooks/goormtextclassificationproject/goormkoreanmrcproject/bigbird_v1.csv', index=False)
sample = sample.rename({'id':'guid'}, axis='columns')

test_data =pd.merge(left=test,right=sample,on='guid')

In [ ]:
test_data

## validation data 에서 성능테스트

In [ ]:

val_features = []
for i, row in val_df.iterrows():
    val_features += prepare_test_features(row, tokenizer)

In [ ]:
val_test_dataset = DatasetRetriever(val_features, mode='test')
val_test_dataloader = DataLoader(
    val_test_dataset,
    batch_size=4, 
    sampler=SequentialSampler(val_test_dataset),
    num_workers=4,
    pin_memory=True, 
    drop_last=False
)

In [ ]:
def get_val_predictions(trained_model):
    model = trained_model.to(device)
    
    start_logits = []
    end_logits = []
    for batch in tqdm(val_test_dataloader):
        with torch.no_grad():
            outputs_start, outputs_end = model(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
            start_logits.append(outputs_start.cpu().numpy().tolist())
            end_logits.append(outputs_end.cpu().numpy().tolist())
            del outputs_start, outputs_end
    gc.collect()
    return np.vstack(start_logits), np.vstack(end_logits)

In [ ]:
start_logits, end_logits = get_val_predictions(trained_model=trained_model)
# Now Calling Function 

fin_preds = postprocess_qa_predictions(val_df, val_features, (start_logits, end_logits))

  0%|          | 0/1277 [00:00<?, ?it/s]

Post-processing 5000 example predictions split into 5108 features.


In [ ]:
from string import punctuation
submission = []
for p1, p2 in fin_preds.items():
    p2 = " ".join(p2.split())
    p2 = p2.strip(punctuation)
    submission.append((p1, p2))
    
sample = pd.DataFrame(submission, columns=["id", "Predicted"])


In [ ]:
val_df['Predicted'] = sample['Predicted']

## levenshtein score 계산

In [ ]:
def levenshtein(s1, s2, debug=False):
    if len(s1) < len(s2):
        return levenshtein(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1]

In [ ]:
for i in range(0, len(val_df)):
  val_df.loc[i, 'levenshtein'] = levenshtein(val_df.loc[i, 'text'], val_df.loc[i, 'Predicted'])

val_df['levenshtein'].mean()

길이를 30으로 했을 때 3.4443521594684388

길이를 20으로 했을 때 3.13953488372093

## Esemble 


In [ ]:
bigbird_v1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/goormtextclassificationproject/goormkoreanmrcproject/bigbird_v1.csv')
bigbird_v2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/goormtextclassificationproject/goormkoreanmrcproject/bigbird_v2.csv')
t5_v6 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/goormtextclassificationproject/goormkoreanmrcproject/KorQuad-T5_v6.csv')

In [ ]:
t5_v6 = t5_v6.rename({'Id':'id'}, axis='columns')
bigbrid_data = pd.merge(bigbird_v1,bigbird_v2,on='id')
predict_data = pd.merge(bigbrid_data,t5_v6,on='id')

In [ ]:
for i in range(len(predict_data)):
    if (predict_data.loc[i, 'Predicted_x'] == predict_data.loc[i, 'Predicted_y']) & (predict_data.loc[i, 'Predicted_x'] == predict_data.loc[i, 'Predicted']):
        predict_data.loc[i, 'result'] = predict_data.loc[i, 'Predicted']
    elif (predict_data.loc[i, 'Predicted_x'] == predict_data.loc[i, 'Predicted_y']) & (predict_data.loc[i, 'Predicted_x'] != predict_data.loc[i, 'Predicted']):
        predict_data.loc[i, 'result'] = predict_data.loc[i, 'Predicted_x']
    elif (predict_data.loc[i, 'Predicted_x'] != predict_data.loc[i, 'Predicted_y']) & (predict_data.loc[i, 'Predicted_x'] == predict_data.loc[i, 'Predicted']):
        predict_data.loc[i, 'result'] = predict_data.loc[i, 'Predicted_x']
    elif (predict_data.loc[i, 'Predicted_x'] != predict_data.loc[i, 'Predicted_y']) & (predict_data.loc[i, 'Predicted_y'] == predict_data.loc[i, 'Predicted']):
        predict_data.loc[i, 'result'] = predict_data.loc[i, 'Predicted']
    elif (predict_data.loc[i, 'Predicted_x'] != predict_data.loc[i, 'Predicted_y']) & (predict_data.loc[i, 'Predicted_y'] != predict_data.loc[i, 'Predicted']):
        predict_data.loc[i, 'result'] = predict_data.loc[i, 'Predicted_y']           
           